In [90]:
using JuMP, GLPK, LinearAlgebra, Combinatorics, Printf

In [153]:
c = [-2; 3; -1; 0; 0; 0]
A = [1 1 1 1 0 0 ;
    4 -3 1 0 1 0 ;
    2 1 -1 0 0 1 ]
b = [10; 3; 10]

3-element Vector{Int64}:
 10
  3
 10

In [140]:
m = Model(GLPK.Optimizer)
@variable(m, x[1:length(c)] >= 0 )
@objective(m, Max, dot(c,x))
@constraint(m, A*x .==b)

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1] + x[2] + x[3] + x[4] == 10.0
 4 x[1] - 3 x[2] + x[3] + x[5] == 3.0
 2 x[1] + x[2] - x[3] + x[6] == 10.0

In [141]:
JuMP.optimize!(m)
obj0 = JuMP.objective_value(m)
opt_x0 = JuMP.value.(x)

6-element Vector{Float64}:
  0.0
 10.0
  7.401486830834377e-16
  0.0
 33.0
  0.0

In [142]:
mutable struct SimplexTableau
    z_c    ::Array{Float64}
    Y      ::Array{Float64}
    x_B    ::Array{Float64}
    obj    ::Float64
    b_idx  ::Array{Int64}
end

In [147]:
z_c = [-2 3 -1 0 0 0]
Y = [1 1 1 1 0 0 ;
    4 -3 1 0 1 0 ;
    2 1 -1 0 0 1]
x_B = [10; 3; 10]
obj = 0 
b_idx = [4,5,6]

3-element Vector{Int64}:
 4
 5
 6

In [148]:
typeof(tableau)

SimplexTableau

In [149]:
fieldnames(typeof(tableau))

(:z_c, :Y, :x_B, :obj, :b_idx)

In [150]:
tableau.b_idx

3-element Vector{Int64}:
 4
 5
 6

In [154]:
function simplex_method(c, A,b)
    tableau = initialize(c, A,b)
    
    while !is_optimal(tableau)
        pivoting!(tableau)
    end
    
    
    opt_x = zeros(length(c))
        
    opt_x[tableau.b_idx] = tableau.x_B
    
    return opt_x, tableau.obj
    
end

simplex_method (generic function with 1 method)

In [155]:
simplex_method(c, A,b)

LoadError: BoundsError: attempt to access 6-element Vector{Float64} at index [1:6, [1, 2, 3]]

In [133]:
c = Array{Float64}(c)

6-element Vector{Float64}:
 -2.0
  3.0
 -1.0
  0.0
  0.0
  0.0

In [134]:
A = Array{Float64}(A)

3×6 Matrix{Float64}:
 1.0   1.0   1.0  1.0  0.0  0.0
 4.0  -3.0   1.0  0.0  1.0  0.0
 2.0   1.0  -1.0  0.0  0.0  1.0

In [135]:
b = Array{Float64}(b)

3-element Vector{Float64}:
 10.0
  3.0
 10.0

In [136]:
m,n = size(A)

(3, 6)

In [107]:
function is_nonnegative(x::Vector)
    return length(x[ x .<0])==0
end

function is_nonnegative(z::Array)
    return length(z[ z .>0])==0
end

is_nonnegative (generic function with 2 methods)

In [108]:
function initial_BFS(A,b)
    m,n = size(A)
    
    comb = collect(combinations(1:n,m))
    for i in length(comb):-1:1
        b_idx = comb[i]
        B = A[:,b_idx]
        x_B = inv(B) * b
        if is_nonnegative(x_B)
        
            return b_idx, x_B, B
        end
    end
    error("Infeasible")
end

initial_BFS (generic function with 1 method)

In [80]:
function print_tableau(t::SimplexTableau)
    m,n = size(t.Y)
    hline0 = repeat("-",6)
    hline1 = repeat("-",7*n)
    hline2 = repeat("-",7)
    hline = join([hline0, "+", hline1, "+", hline2] )
    
    print(hline)
    
    @printf("%6s|","")
    for j in 1:length(t.z_c)
        @printf("%6.2f",t.z_c[j])
    end
    @printf("| %6.2f\n",t.obj)
    
    println(hline)
    
    for i in 1:m
        @printf("x[%jd]  |", t.b_idx[i])
        for j in 1:n
            @printf("%6.2f", t.Y[i,j])
        end
        @printf("| %6.2f\n", t.x_B[I])
    end
    
    println(hline)
end

print_tableau (generic function with 1 method)

In [109]:
function initialize(c,A,b)
    c = Array{Float64}(c)
    A = Array{Float64}(A)
    b = Array{Float64}(b)
    
    m,n = size(A)
    
    b_idx, x_B, B = initial_BFS(A,b)
    Y = inv(B) *A
    c_B = c[b_idx]
    obj = dot(c_B, x_B)
    
    z_c = zeros(1,n)
    n_idx = setdiff(1:n, b_idx)
    z_c[n_idx] = c_B'* inv(B) * A[:,n_idx] - c[:,n_idx]'
    
    return SimplexTableau(z_c, Y, x_B, obj, b_idx)
end

initialize (generic function with 1 method)

In [110]:
function is_optimal(t::SimplexTableau)
    return is_nonpositive(t.z_c)
end

function is_nonpositive(z::Array)
    return length(z[z .>0]) ==0
end

is_nonpositive (generic function with 1 method)

In [111]:
function pivoting!(t::SimplexTableau)
    m,n = size(t.Y)
    entering, exiting = pivoting_point(t)
    
    println("pivoting: entering = x_$entering, exiting = x_$(t.b_idx[exiting])")
    
    coef = t.Y[exiting, entering]
    t.Y[exiting,:] /= coef 
    t.x_B[exiting] /= coef
    
    for i in setdiff(1:m, exiting)
        coef = t.Y[i, entering]
        t.Y[i,:] -= coef* t.Y[exiting,:]
        t.x_B[i] -= coef *t.x_B[exiting]
    end
    
    coef = t.z[entering]
    t.z_c -= coef* t.Y[exiting,:]
    t.obj -= coef *t.x_B[exiting]
    
    t.b_idx[findfirst(t.b_idx .== t.b_idx[exiting])       ] = entering
    
end

pivoting! (generic function with 1 method)

In [112]:
function pivot_point(t::SimplexTableau)
    entering = argmax(t.z_c)
    if exiting == 0 
        error("optimal")
    end
    
    pos_idx = findall(t.Y[:, entering] .> 0 )
    if length(pos_idx) == 0 
        error("unbounded")
    end    
    exiting = pos.idx[argmin(t.x_B[pos_idx]  ./ t.Y[pos_idx,entering]         )]
    
    return entering, exiting
    
end

pivot_point (generic function with 1 method)

In [145]:
include("simplex_method.jl")

using Main.SimplexMethod

In [156]:
initial_BFS(A,b)

([4, 5, 6], [10.0, 3.0, 10.0], [1 0 0; 0 1 0; 0 0 1])

In [161]:
print_tableau()

LoadError: MethodError: no method matching print_tableau(::Matrix{Int64})
[0mClosest candidates are:
[0m  print_tableau([91m::SimplexTableau[39m) at In[80]:1